# Created by Hamza Mughal

some of the examples model generated

(input before hyphen - output afte hyphen)

"""

I am a - I am a humble man who is here to learn from my friends in the new world

you dont - you dont know what its like to carry a child and give birth

he - he made small talk

his - his body was discovered in an abandoned theater

her - her room was empty and the bed was still neatly made for her

I am sorry - I am sorry but we have to ask you to be out by the end of the week

I apologize - I apologize for interrupting like this

do - do you know what time it is

you know that - you know that its a bad thing for me

fine - fine ill do it my dear man

how to - how to find out this thing you dont know

"""

# Load dataset. Kaggle 'movie-subtitle-dataset'

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adiamaan/movie-subtitle-dataset")

100%|██████████| 243M/243M [00:01<00:00, 209MB/s]

Extracting files...


In [ ]:
file_path = path + "/movies_subtitles.csv"

In [ ]:
import pandas as pd

# read only 'text' column and 50k records only (to avoid memory hit)
df = pd.read_csv(file_path, usecols=['text'],nrows=50000)

# Pre-processing

In [ ]:
# remove unecessory characters
df['text'] = df['text'].str.replace(r'[^\w\s]', '', regex=True)
df['text'] = df['text'].str.replace(r'<.*?>', '', regex=True)
df['text'] = df['text'].str.replace(r'\n', ' ', regex=True)

Tokenizing data

In [ ]:
# Breaking sentences/phrases into sequences using nltk E.g. "this is a sentence" -> ["this","is","a","sentence"]
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab') # Download the 'punkt_tab' data package

# Ensure the 'text' column contains only strings
df['text'] = df['text'].astype(str)

df['sequence'] = df['text'].apply(word_tokenize)
df = df['sequence']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Craete input and output sequences

In [ ]:
# create sequences E.g.
# column (sequence)
# ["this","is","a","sentence"]
# ->
# column (sequence)     column (output)
# ["this","is","a"]     ["sentence"]
# ["this","is"]         ["a"]
# ["this"]              ["is"]

def process_sequences(df):
    """
    Processes sequences to create input-output pairs for language modeling.

    This function takes a DataFrame where each row contains a sequence of tokens
    (e.g., words in a sentence) and transforms it into a new DataFrame suitable
    for training a language model.

    For each sequence in the input DataFrame, it creates multiple input-output pairs.
    The input is a subsequence of the original sequence, and the output is the
    next word in the original sequence.

    Args:
        df (pandas.DataFrame): A DataFrame where each row contains a sequence of tokens.
                                 It is assumed that the sequences are stored in a column
                                 named 'sequence'.

    Returns:
        pandas.DataFrame: A new DataFrame with two columns: 'sequence' and 'output'.
                           Each row represents an input-output pair for the language model.
    """
    new_rows = []  # Initialize an empty list to store the new rows
    for sequence in df:  # Iterate through each sequence in the input DataFrame
        for i in range(1, len(sequence)):  # Iterate through different lengths of subsequences
            new_rows.append({  # Create a new row for each subsequence
                'sequence': sequence[:-i],  # Input: Subsequence from the beginning to (len - i)
                'output': sequence[-i]     # Output: The next word in the original sequence
            })
    return pd.DataFrame(new_rows)  # Convert the list of new rows into a DataFrame

# Processed dataframe
result_df = process_sequences(df)

In [ ]:
# convert pandas sequence into datafram
df = df.to_frame(name='sequence')

In [ ]:
result_df

,sequence,output
0,"[BOY, All, right, everyone, This, is, a]",stickup
1,"[BOY, All, right, everyone, This, is]",a
2,"[BOY, All, right, everyone, This]",is
3,"[BOY, All, right, everyone]",This
4,"[BOY, All, right]",everyone
...,...,...
239002,"[Starting, now, Ill, never, compose]",another
239003,"[Starting, now, Ill, never]",compose
239004,"[Starting, now, Ill]",never
239005,"[Starting, now]",Ill


Create tokenizer/dictionary of your dataset

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Initialize the Tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on your corpus (i.e., all the text in your dataset)
tokenizer.fit_on_texts(df["sequence"])
del df

# # Convert the text sequences to integer sequences
input_sequences = tokenizer.texts_to_sequences(result_df['sequence'])
output_words = tokenizer.texts_to_sequences(result_df['output'])
del result_df

In [ ]:
# flatting output sequence
output_words = [item[0] for item in output_words]

Add padding to input sequence to make all input sequence same lenght, to allow parallel training

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Determine the maximum sequence length
max_sequence_length = max(len(seq) for seq in input_sequences)

# Pad the input sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
del input_sequences

Create one-hot encoding of your output sequence to allow model to do probablilty distribution using softmax

In [ ]:
from tensorflow.keras.utils import to_categorical

# Number of unique tokens in your vocabulary
vocab_size = len(tokenizer.word_index) + 1  # +1 because tokenizer indices are 1-based

# Convert to one-hot encoded format
output_words_categorical = to_categorical(output_words, num_classes=vocab_size)

# GLOVE embeddings if want to use

In [ ]:
# download and unzip embeddings
!wget https://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove.42B.300d.zip

--2025-01-22 14:34:37--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2025-01-22 14:34:37--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip.1’

glove.42B.300d.zip.   2%[                    ]  52.63M  5.12MB/s    eta 3m 28s ^C
Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      

In [2]:
def create_embedding_matrix(vocab_size, tokenizer, embedding_dim):
  """
  Creates an embedding matrix using pre-trained GloVe embeddings.

  Args:
      vocab_size (int): The size of the vocabulary.
      tokenizer (keras.preprocessing.text.Tokenizer): The tokenizer used to create the vocabulary.
      embedding_dim (int): The dimensionality of the GloVe embeddings.

  Returns:
      numpy.ndarray: The embedding matrix.
  """
  vocabulary = tokenizer.word_index.keys()  # Get all the words in the vocabulary
  path_to_embeddings = '/content/glove.42B.300d.txt'  # Path to the GloVe embeddings file

  # Load GloVe embeddings into a dictionary, where keys are words and values are embedding vectors
  embeddings_index = {}
  with open(path_to_embeddings, encoding='utf-8') as f:
      for line in f:
          values = line.split()  # Split the line into word and embedding values
          word = values[0]  # The first value is the word
          if word in vocabulary:  # Only consider words that are in our vocabulary
            coefs = np.asarray(values[1:], dtype='float32')  # Convert embedding values to a NumPy array
            embeddings_index[word] = coefs  # Store the embedding vector for the word

  # Create the embedding matrix
  embedding_matrix = np.zeros((vocab_size, embedding_dim))  # Initialize with zeros
  for word, i in tokenizer.word_index.items():  # Iterate through the vocabulary
      embedding_vector = embeddings_index.get(word)  # Get the embedding vector for the word
      if embedding_vector is not None:  # If an embedding vector was found for the word
          embedding_matrix[i] = embedding_vector  # Store it in the embedding matrix

  return embedding_matrix  # Return the created embedding matrix

# Create model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

# Set up model parameters
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token
hidden_units = 512  # Number of units in the LSTM layer
embedding_dim = 300  # Match the GloVe dimension, if using
use_glove_embeddings = False

# Build the model
model = Sequential()

if use_glove_embeddings:
  embedding_matrix = create_embedding_matrix(tokenizer, embedding_dim)
  # 1. Embedding layer: Convert token IDs to dense vectors
  model.add(Embedding(
      input_dim=vocab_size,
      output_dim=embedding_dim,
      weights=[embedding_matrix],
      trainable=True  # Set to False to keep GloVe embeddings fixed
  ))
else:
  # 1. Embedding layer: Convert token IDs to dense vectors
  model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))

# 2. LSTM layer: Capture sequential dependencies
model.add(LSTM(units=hidden_units))

# 3. Dense layer: Output a distribution over possible next words
model.add(Dense(vocab_size, activation='softmax'))  # Softmax to generate a probability distribution

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │       2,618,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,618,400 (9.99 MB)

 Trainable params: 2,618,400 (9.99 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
del output_words

# Train the model

In [ ]:
# Train the model
model.fit(padded_input_sequences, output_words_categorical, epochs=15, batch_size=64)

Epoch 1/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 42s 13ms/step - accuracy: 0.0744 - loss: 6.5237
Epoch 2/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 38s 13ms/step - accuracy: 0.1530 - loss: 5.1098
Epoch 3/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - accuracy: 0.2145 - loss: 4.1865
Epoch 4/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 40s 13ms/step - accuracy: 0.3282 - loss: 3.2543
Epoch 5/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - accuracy: 0.4581 - loss: 2.5246
Epoch 6/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - accuracy: 0.5635 - loss: 2.0018
Epoch 7/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - accuracy: 0.6447 - loss: 1.6220
Epoch 8/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - accuracy: 0.6963 - loss: 1.3800
Epoch 9/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.7299 - loss: 1.2244
Epoch 10/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.7503 - loss: 1.1225
Epoch 11/20
2649/2649 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - accuracy: 0.7636 - loss: 1.0563
Epoch 12

# Save model

In [ ]:
import pickle

# save model
with open('predictify-model.pkl', 'wb') as f:
    pickle.dump({'model': model, 'tokenizer': tokenizer}, f)

# Use model to generate text

In [9]:
import numpy as np

def generate_sentence(input_sequence, model, tokenizer, num_words=10):
    """
    Generates a sentence by predicting the next words based on a given starting sequence.

    Args:
        input_sequence (list): The starting sequence of words.
        model: The trained language model.
        tokenizer: The tokenizer used to process the text.
        num_words (int): The number of additional words to generate.

    Returns:
        str: The generated sentence.
    """

    input_sequence = input_sequence.split()

    # Ensure input_sequence is a list
    if not isinstance(input_sequence, list):
        input_sequence = [input_sequence]

    for _ in range(num_words):
        input_sequence_tokenized = tokenizer.texts_to_sequences([input_sequence])
        input_sequence_tokenized = np.array(input_sequence_tokenized[0])
        input_sequence_tokenized = input_sequence_tokenized.reshape(1, -1)

        predicted_probabilities = model.predict(input_sequence_tokenized)
        predicted_word_index = predicted_probabilities.argmax(axis=-1)
        predicted_word = tokenizer.index_word[predicted_word_index[0]]

        input_sequence.append(predicted_word)

    return " ".join(input_sequence)

In [ ]:
import pickle

# Load the pre-trained model and tokenizer from the  pickle file
with open('predictify-model.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
    model = loaded_data['model']
    tokenizer = loaded_data['tokenizer']

In [ ]:
# Example usage:
sentence = generate_sentence("enter your prompt", model, tokenizer, num_words=15)
print(sentence)